In [5]:
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import requests
import pprint
import random
import pickle

In [78]:
# 시작일,종료일 설정
start = "2022-09-01"
last = "2022-09-15"

# 시작일, 종료일 datetime 으로 변환
start_date = datetime.strptime(start, "%Y-%m-%d")
last_date = datetime.strptime(last, "%Y-%m-%d")

# 종료일 까지 반복
date_time = []
cnt = 0
while start_date <= last_date:
    date = start_date.strftime("%Y-%m-%d")
    date_time.append(date)
    cnt += 1
    # 하루 더하기
    start_date += timedelta(days=1)

In [253]:
data = pd.read_csv("고기데이터.tsv", sep='\t')
data

,카테고리,거래일자,가격,단량,단위,상점,대분류,도소매,품목명
0,오징어,2022-04-28,0,1,개,이마트 은평점,17,1,오징어
1,오징어,2022-04-28,0,1,개,이마트 은평점,17,1,오징어
2,오징어,2022-04-28,0,1,개,이마트 은평점,17,1,오징어
3,오징어,2022-04-28,2500,1,개,이마트 은평점,17,1,오징어
4,오징어,2022-05-26,4580,1,개,이마트 은평점,17,1,오징어
...,...,...,...,...,...,...,...,...,...
22195,닭고기,2021-12-30,8000,1000,g,둔촌역전통시장,16,1,닭고기
22196,쇠고기,2021-12-30,46200,100,g,이마트 명일점,16,1,쇠고기
22197,돼지고기,2021-12-30,16980,100,g,이마트 명일점,16,1,돼지고기
22198,쇠고기,2021-12-30,37800,100,g,홈플러스 강동점,16,1,쇠고기


In [62]:
df = pd.DataFrame(columns = {'카테고리', '거래일자', '가격', '단량', '단위', '상점', '대분류', '도소매', '품목명'})

In [44]:
'이마트 은평점'.strip(' ')

'이마트 은평점'

In [256]:
new_store = ['이마트', '홈플러스', '롯데마트', '서울중앙시장', '남대문시장',
 '금남시장',
 '뚝도시장',
 '노룬산골목시장',
 '돈암제일시장',
 '숭인시장',
 '수유재래시장',
 '상계중앙시장',
 '공릉동도깨비시장',
 '송화시장',
 '고척근린시장',
 '남구로시장',
 '대림중앙시장',
 '남성시장',
 '신영시장',
 '목3동시장','청담삼익시장',
 '도곡시장',
 '영천시장',
 '인왕시장',
 '경동시장',
 '청량리종합시장',
 '둔촌역전통시장',
 '암사종합시장',
 '원당종합시장',
 '자양골목시장',
 '우림시장',
 '동원시장',
 '영동시장',
 '방이시장',
 '화곡본동시장','롯데백화점', '망원시장',
 '신창시장', '현대백화점', '신세계백화점', 'NC백화점',
             '방배종합시장',
 '대림시장',
 '마포농수산물시장',
 '뉴코아아울렛',
 '마천중앙시장',
 '대조시장',
 '공릉동 도깨비시장',
 '광장시장',
 '후암시장',
 '용문시장',
 '통인시장',
 '현대시장',

    ]
new_store = list(set(new_store))
new_store

['현대백화점',
 '공릉동도깨비시장',
 '신영시장',
 '송화시장',
 'NC백화점',
 '인왕시장',
 '남성시장',
 '현대시장',
 '숭인시장',
 '자양골목시장',
 '경동시장',
 '서울중앙시장',
 '공릉동 도깨비시장',
 '신창시장',
 '원당종합시장',
 '통인시장',
 '영동시장',
 '화곡본동시장',
 '우림시장',
 '대림중앙시장',
 '이마트',
 '마포농수산물시장',
 '금남시장',
 '롯데마트',
 '신세계백화점',
 '도곡시장',
 '대림시장',
 '상계중앙시장',
 '남구로시장',
 '암사종합시장',
 '노룬산골목시장',
 '뉴코아아울렛',
 '마천중앙시장',
 '남대문시장',
 '영천시장',
 '둔촌역전통시장',
 '방이시장',
 '동원시장',
 '수유재래시장',
 '용문시장',
 '청량리종합시장',
 '돈암제일시장',
 '망원시장',
 '방배종합시장',
 '고척근린시장',
 '목3동시장',
 '청담삼익시장',
 '후암시장',
 '홈플러스',
 '뚝도시장',
 '롯데백화점',
 '대조시장',
 '광장시장']

In [255]:
store = list(data['상점'].unique())

['이마트 은평점',
 '이마트 용산점',
 '이마트 상봉점',
 '이마트 이수점',
 '이마트 명일점',
 '이마트 미아점',
 '이마트 역삼점',
 '이마트 성수점',
 '이마트 왕십리점',
 '이마트 자양점',
 '이마트 창동점',
 '이마트 목동점',
 '이마트 가양점',
 '이마트 신도림점',
 '이마트 여의도점',
 '홈플러스 동대문점',
 '홈플러스 면목점',
 '홈플러스 방학점',
 '홈플러스 중계점',
 '홈플러스 월드컵점',
 '홈플러스 영등포점',
 '홈플러스 잠실점',
 '홈플러스 강동점',
 '롯데마트 서울역점',
 '롯데마트 강변점',
 '서울중앙시장',
 '남대문시장',
 '금남시장',
 '뚝도시장',
 '노룬산골목시장',
 '돈암제일시장',
 '숭인시장',
 '수유재래시장',
 '상계중앙시장',
 '공릉동도깨비시장',
 '송화시장',
 '고척근린시장',
 '남구로시장',
 '대림중앙시장',
 '남성시장',
 '신영시장',
 '목3동시장',
 '홈플러스목동점',
 '청담삼익시장',
 '도곡시장',
 '영천시장',
 '인왕시장',
 '경동시장',
 '청량리종합시장',
 '둔촌역전통시장',
 '암사종합시장',
 '원당종합시장',
 '자양골목시장',
 '우림시장',
 '동원시장',
 '영동시장',
 '방이시장',
 '화곡본동시장',
 '롯데백화점 강남점',
 '롯데백화점 잠실점',
 '망원시장',
 '신창시장',
 '롯데백화점 미아점',
 '롯데백화점 영등포점',
 '현대백화점 신촌점',
 '신세계백화점 강남점',
 'NC백화점 불광점',
 'NC백화점 신구로점',
 '방배종합시장',
 '대림시장',
 '마포농수산물시장',
 '뉴코아아울렛 강남점',
 '마천중앙시장',
 '대조시장',
 '롯데백화점 청량리점',
 '공릉동 도깨비시장',
 '광장시장',
 '후암시장',
 '용문시장',
 '현대백화점 미아점',
 '롯데백화점 노원점',
 '통인시장',
 '현대시장',
 '롯데백화점 관악점']

In [261]:
meats = ['닭고기','돼지고기']

In [264]:
meat_date = []
for mart in tqdm(new_store):
    for meat in meats:
        date = '2022-09'
        for start in tqdm(range(1, 50000, 1000)):
            end = start + 999
            url = f'http://openAPI.seoul.go.kr:8088/58744371596c69763838426259716d/json/ListNecessariesPricesService/{start}/{end}/{mart}/{meat}/{date}'
            response = requests.get(url).json()
            
            if response.get('ListNecessariesPricesService'):
                rows = response.get('ListNecessariesPricesService').get('row')
                for row in rows:
                    meat_date.append(row)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.88it/s]

 54%|████████████████████████████████████████████▎                                     | 27/50 [00:05<00:04,  4.80it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.95it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.72it/s]

 10%|████████▎                                                                          | 5/50 [00:01<00:09,  4.55it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.76it/s]

 68%|███████████████████████████████████████████████████████▊                          | 34/50 [00:06<00:03,  4.96it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.91it/s]

 22%|██████████████████                                                                | 11/50 [00:02<00:07,  5.16it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.79it/s]

 80%|█████████████████████████████████████████████████████████████████▌                | 40/50 [00:08<00:02,  4.95it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.68it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.93it/s]

 38%|███████████████████████████████▏                                                  | 19/50 [00:03<00:06,  4.88it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.02it/s]

 96%|██████████████████████████████████████████████████████████████████████████████▋   | 48/50 [00:10<00:00,  5.15it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.92it/s]

 50%|█████████████████████████████████████████                                         | 25/50 [00:04<00:04,  5.04it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.14it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.25it/s]

  6%|████▉                                                                              | 3/50 [00:00<00:09,  5.10it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [17:50<00:00, 20.20s/it]


In [58]:
with open('data.pickle', 'wb') as f:
    pickle.dump(meat_date, f, pickle.HIGHEST_PROTOCOL)

In [266]:
category = []
datetime = []
price = []
quantity = []
unit = []
store = []
high = []
name = []
for d in meat_date:
    category.append(d['A_NAME'])
    datetime.append(d['P_DATE'])
    price.append(d['A_PRICE'])
    quantity.append(d['A_UNIT'])
    unit.append(d['A_UNIT'])
    store.append(d['M_NAME'])
    name.append(d['A_NAME'])

In [267]:
df = pd.DataFrame()
df['카테고리'] = category
df['거래일자'] = datetime
df['가격'] = price
df['단량'] = quantity
df['단위'] = unit
df['상점'] = store
df['품목명'] = name

In [276]:
for s in store:
    price = df[(df['상점'] == s) & ((df['품목명'] == '돼지고기(생삼겹살)')|(df['품목명'] == '돼지고기(삼겹살)')|(df['품목명'] == '돼지고기'))]['가격'].tolist()
    if price:
        for dtime in date_time:
            check = {}
            check['카테고리'] = '돼지고기'
            check['거래일자'] = dtime
            check['가격'] = random.choice(price)
            check['단량'] = 100
            check['단위'] = 'g'
            check['상점'] = s
            check['품목명'] = '돼지고기'
            check['대분류'] = 16
            ingredient.append(check)

In [277]:
for s in store:
    price = df[(df['상점'] == s) & ((df['품목명'] == '돼지고기(생삼겹살)')|(df['품목명'] == '돼지고기(삼겹살)')|(df['품목명'] == '돼지고기'))]['가격'].tolist()
    if price:
        for dtime in date_time2:
            check = {}
            check['카테고리'] = '돼지고기'
            check['거래일자'] = dtime
            check['가격'] = random.choice(price)
            check['단량'] = 100
            check['단위'] = 'g'
            check['상점'] = s
            check['품목명'] = '돼지고기'
            check['대분류'] = 16
            ingredient.append(check)

In [282]:
category = []
datetime = []
price = []
quantity = []
unit = []
store = []
high = []
so = []
name = []
for d in ingredient:
    category.append(d['카테고리'])
    datetime.append(d['거래일자'])
    price.append(d['가격'])
    quantity.append(d['단량'])
    unit.append(d['단위'])
    store.append(d['상점'])
    high.append(d['대분류'])
    so.append(1)
    name.append(d['품목명'])

In [285]:
new_df = pd.DataFrame()
new_df['카테고리'] = category
new_df['거래일자'] = datetime
new_df['가격'] = price
new_df['단량'] = quantity
new_df['단위'] = unit
new_df['상점'] = store
new_df['대분류'] = high
new_df['도소매'] = so
new_df['품목명'] = name

In [286]:
new_df

,카테고리,거래일자,가격,단량,단위,상점,대분류,도소매,품목명
0,오징어,2022-09-01,5280,1,마리,이마트 은평점,17,1,오징어
1,오징어,2022-09-02,4380,1,마리,이마트 은평점,17,1,오징어
2,오징어,2022-09-03,4380,1,마리,이마트 은평점,17,1,오징어
3,오징어,2022-09-04,4380,1,마리,이마트 은평점,17,1,오징어
4,오징어,2022-09-05,6980,1,마리,이마트 은평점,17,1,오징어
...,...,...,...,...,...,...,...,...,...
63175,돼지고기,2022-09-26,3167,100,g,광장시장,16,1,돼지고기
63176,돼지고기,2022-09-27,2833,100,g,광장시장,16,1,돼지고기
63177,돼지고기,2022-09-28,3167,100,g,광장시장,16,1,돼지고기
63178,돼지고기,2022-09-29,3167,100,g,광장시장,16,1,돼지고기


In [291]:
new_df.to_csv('마지막.tsv', sep='\t', index=False)